<a href="https://colab.research.google.com/github/Shashank1202/Deep_Learning/blob/main/LLMs/BERT_for_Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implemtation of BERT Model for Masking**

In [2]:
from transformers import pipeline

pipe= pipeline("fill-mask", model="google-bert/bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model= AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
tokenizer= AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
inputs= tokenizer("The capital of France is [MASK].",return_tensors= "pt")

In [6]:
import torch

with torch.no_grad():
  logits= model(**inputs).logits

In [9]:
mask_token_index= (inputs.input_ids== tokenizer.mask_token_id)[0].nonzero(as_tuple= True)[0]

In [10]:
predicted_token_id= logits[0, mask_token_index].argmax(axis= -1)
tokenizer.decode(predicted_token_id)

'paris'

In [12]:
labels= tokenizer("The capital of France is Paris.", return_tensors= "pt")["input_ids"]

In [13]:
labels= torch.where(inputs.input_ids== tokenizer.mask_token_id, labels, -100)

In [14]:
outputs= model(**inputs, labels= labels)
round(outputs.loss.item(), 2)

0.88

**BERT Model Implementation with TensorFlow**

In [18]:
from transformers import TFBertForMaskedLM
import tensorflow as tf

model= TFBertForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [29]:
inputs= tokenizer("The capital of France is [MASK]", return_tensors= "tf")
logits= model(**inputs).logits

In [30]:
mask_token_index= tf.where((inputs.input_ids==tokenizer.mask_token_id)[0])
selected_logits= tf.gather_nd(logits[0], indices= mask_token_index)

In [31]:
predicted_token_id= tf.math.argmax(selected_logits, axis= -1)
tokenizer.decode(predicted_token_id)

'.'

In [33]:
labels= tokenizer("The capital of France is Paris", return_tensors="tf")["input_ids"]
labels= tf.where(inputs.input_ids==tokenizer.mask_token_id, labels, -100)

In [34]:
outputs= model(**inputs, labels= labels)
round(float(outputs.loss), 2)

12.24

**Flax BERT For Masking**

In [36]:
from transformers import FlaxBertForMaskedLM

In [37]:
model= FlaxBertForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

flax_model.msgpack:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [38]:
inputs= tokenizer("The capital of France is [MASK].", return_tensors= "jax")

In [43]:
outputs= model(**inputs)

In [40]:
logits= outputs.logits

In [45]:
import jax.numpy as jnp

masked_token_index= jnp.where(inputs["input_ids"]== tokenizer.mask_token_id)[1].item()

In [47]:
predicted_token_id= jnp.argmax(logits[0, masked_token_index])

In [48]:
predicted_token= tokenizer.decode(predicted_token_id)

In [50]:
predicted_token

paris
